# Carga dos dados na camada gold

In [0]:
from pyspark.sql.functions import col, year, month, count, sum, avg, current_date, current_timestamp,unix_timestamp, when, lit

anos = ParamAno
meses = ParamMes
df_gold = None

df_silver =  spark.read.table("table_silver") 
df_silver = df_silver.withColumn("duracao_viagens", unix_timestamp(col("tpep_dropoff_datetime"))- unix_timestamp(col("tpep_pickup_datetime"))/60)
df_gold = (
df_silver
.withColumn("Ano", year("tpep_pickup_datetime"))
.withColumn("Mes", month("tpep_pickup_datetime"))
.withColumn("Nome_Mes", when(month(col("tpep_pickup_datetime")) == 1, lit("Janeiro"))
.when(month(col("tpep_pickup_datetime")) == 2, lit("Fevereiro"))
.when(month(col("tpep_pickup_datetime")) == 3, "Março")
.when(month(col("tpep_pickup_datetime")) == 4, "Abril")
.when(month(col("tpep_pickup_datetime")) == 5, "Maio")
.when(month(col("tpep_pickup_datetime")) == 6, "Junho")
.when(month(col("tpep_pickup_datetime")) == 7, "Julho")
.when(month(col("tpep_pickup_datetime")) == 8, "Agosto")
.when(month(col("tpep_pickup_datetime")) == 9, "Setembro")
.when(month(col("tpep_pickup_datetime")) == 10, "Outubro")
.when(month(col("tpep_pickup_datetime")) == 11, "Novembro")
.when(month(col("tpep_pickup_datetime")) == 12, "Dezembro"))
.groupBy("Ano", "Mes","Nome_Mes")
.agg(
    count("*").alias("Total_Corridas"),
    sum("total_amount").cast("decimal(18,2)").alias("Receita_Total"),
    avg("trip_distance").cast("decimal(18,2)").alias("Media_Distancia"),
    avg("passenger_count").cast("decimal(18,2)").alias("Media_Passageiros"),                
    sum("fare_amount").cast("decimal(18,2)").alias("Total_Tarifas"),
    avg("duracao_viagens").cast("decimal(18,2)").alias("Media_Tempo_Viagem")
)
# Adiciona colunas de auditoria
.withColumn("Dt_carga_gold", current_date())
.withColumn("Ts_carga_gold", current_timestamp())
)

df_gold.write.mode('overwrite').format("delta").saveAsTable("table_gold") 
   

### Rank dos 3 meses com maior receita por ano

In [0]:
%sql
SELECT 
    Ano,
    Nome_Mes,
    Receita_Total,
    RANK() OVER (PARTITION BY Ano ORDER BY Receita_Total DESC) AS Ranking
FROM table_gold
QUALIFY RANK() OVER (PARTITION BY Ano ORDER BY Receita_Total DESC) <= 3
ORDER BY Ano, Ranking;

Ano,Nome_Mes,Receita_Total,Ranking
2001,Janeiro,171.70,1
2002,Dezembro,117.25,1
2003,Janeiro,209.45,1
2008,Dezembro,264.95,1
2009,Janeiro,16.40,1
2014,Novembro,52.38,1
2022,Dezembro,658.23,1
2022,Outubro,332.26,2
2023,Março,89371940.14,1
2023,Janeiro,78382549.59,2


###  **Receita_por_Corrida** Calcula a Receita  por corrida considerando a receita total e o total da corrida

###  **Tarifa_por_Corrida** Calcula a Tarifa por corrida considerando a tarifas total e o total da corrida


In [0]:
%sql
SELECT 
    Ano,    
    Nome_Mes,    
    cast(Receita_Total / Total_Corridas as decimal(18,2)) AS Receita_por_Corrida,
    cast(Total_Tarifas / Total_Corridas as decimal(18,2)) AS Tarifa_por_Corrida
   
FROM table_gold
ORDER BY Ano, mes;

Ano,Nome_Mes,Receita_por_Corrida,Tarifa_por_Corrida
2001,Janeiro,57.23,47.97
2002,Dezembro,58.63,50.85
2003,Janeiro,69.82,64.13
2008,Dezembro,37.85,30.17
2009,Janeiro,16.40,11.40
2014,Novembro,52.38,33.10
2022,Outubro,55.38,48.17
2022,Dezembro,26.33,16.63
2023,Janeiro,26.97,18.26
2023,Fevereiro,26.89,18.14


###  **Receita_por_Corrida** Calcula a Receita  por corrida considerando a receita total e o total da corrida

###  **Tarifa_por_Corrida** Calcula a Tarifa por corrida considerando a tarifas total e o total da corrida
### **Receita_liquido_corrida** leva em consideração a receita e o valor das tarifas

In [0]:
%sql
select
Receita_por_Corrida,
Tarifa_por_Corrida,
 Receita_por_Corrida -Tarifa_por_Corrida as Receita_liquido_corrida
from (
SELECT 
    Ano,    
    Nome_Mes,
    mes,    
    cast(Receita_Total / Total_Corridas as decimal(18,2)) AS Receita_por_Corrida,
    cast(Total_Tarifas / Total_Corridas as decimal(18,2)) AS Tarifa_por_Corrida
   
FROM table_gold
)
ORDER BY Ano, mes;

Receita_por_Corrida,Tarifa_por_Corrida,Receita_liquido_corrida
57.23,47.97,9.26
58.63,50.85,7.78
69.82,64.13,5.69
37.85,30.17,7.68
16.40,11.40,5.00
52.38,33.10,19.28
55.38,48.17,7.21
26.33,16.63,9.70
26.97,18.26,8.71
26.89,18.14,8.75
